In [1]:
import json
from typing import Literal
from openai import OpenAI
from pydantic import BaseModel, Field
import pandas as pd
import openai
import time

from tqdm import tqdm

In [ ]:
api_key = '<api–key>'

In [3]:
client = OpenAI(api_key=api_key)

In [5]:
input="""**Задача:**
На основе предоставленного JSON-документа с параметрами глазного дна составь развёрнутое текстовое заключение. Ориентируйся на стиль и структуру примера, стремясь к естественной, живой и грамотной формулировке текста. При этом соблюдай строгость в медицинских терминах.

**Формат:**

* Описание должно быть текстовым, структурированным и последовательным, как в приведённом примере.
* Допускается совместное описание артерий и вен, если у них одинаковые параметры (Пример 1). Если параметры разные, их необходимо описывать отдельно (Пример 2).
* Не надо включать в текстовое описание пояснения из скобок в текстах патологий.

**Данные:**
JSON-документ:

```
{
"ДЗН": {
    "Цвет": "Бледно-Розовый",
    "Монотонность": "Наблюдается",
    "Размер": "Меньше нормы",
    "Форма": "Не правильная",
    "Границы": "Размытые",
    "Экскавация": {
        "Размер": Меньше нормы,
        "Сектор": В центре
    },
    "Э/Д": 0.4,
    "Сосудистый пучок": В центре,
    "Патология ДЗН",: К наружи от экскавации темно-серый очаг с четкими границами 0.2 ДД., 
},
"Сосуды": {
    "Артерии": {
        "Ход": Нормальный,
        "Извитость": Нормальная,
        "Бифуркация": Нормальная,
        "Калибр": Нормальный
    },
    "Вены": {
        "Ход": Нормальный,
        "Извитость": Нормальная,
        "Бифуркация": Нормальная,
        "Калибр": Нормальный
    },
    "А/В индекс": 2:3,
    "Патология Сосудов": По ходу сосудов единичные шарообразные утолщения стенок сосудов, окружённое очагами красного цвета с четкими границами (кровоизлияния),
},
"Макула": {
    "Макулярный рефлекс": Нормальный,
    "Фовеальный рефлекс": Нормальный,
    "Патология Макулы": Дополнительно присутствуют кольцевидные рефлексы (кистовидный отёк).,
},
"Переферия": {
    "Патология Переферии": По всему глазному дну множественные очаги красного цвета (кровоизлияния).,
}
}
```

**Пример 1:**
ДЗН бледно-розовый, монотонность не наблюдается, диск нормального размера и правильной формы, границы диска четкие. Экскавация диска нормального размера, расположена в центре, соотношение э/д = 0.3. Сосудистый пучок расположен в центре. ДЗН отечен, выступает в стекловидное тело, границы стерты.
Сосуды имеют нормальный ход и нормальную извитость, бифуркация и калибр в норме, соотношение калибра артерий и вен 2/3. Штопорообразная извитость мелких артериол на переферии сетчатки.
Макулярный и фовеолярные рефлексы четкие. Макула отечна, границы нечёткие, присутствует звездчатая фигура из твердых экссудатов.
По периферии множественные серовато-белые очаги хориоретинальной атрофии с четкими контурами.

**Пример 2:**
ДЗН серый, монотонный, границы размытые, овальной формы, увеличенного размера. Экскавация расширена, в центре, соотношение э/д = 0.6. Сосудистый пучок расположен центрально. К наружи от экскавации темно-серый очаг с четкими границами 0.2 ДД.
Артерии смещены наружу, извитые, бифуркация под острым углом, калибр расширен. Вены имеют нормальный ход, извитость нормальная, бифуркация нормальная, калибр расширенный, соотношение А/В = 1/1. По ходу сосудов единичные шарообразные утолщения стенок сосудов, окружённое очагами красного цвета с четкими границами.
Макулярный рефлекс сглаженный, фовеолярный отсутствует. Дополнительно присутствуют кольцевидные рефлексы.
По всему глазному дну множественные очаги красного цвета.

**Твоя задача:**
Создай полное текстовое описание глазного дна по JSON-документу, следуй телеграфному стилю, правилам формата и примерам.
"""

In [8]:
model_str="gpt-4.5-preview-2025-02-27"

In [9]:
response = client.responses.create(
    model=model_str,
    input=input
)

print(response.output_text)

ДЗН бледно-розового цвета, монотонность наблюдается, диск уменьшенного размера и неправильной формы с размытыми границами. Экскавация уменьшенного размера, расположена центрально, соотношение э/д = 0.4. Сосудистый пучок располагается центрально. Кнаружи от экскавации определяется темно-серый очаг с четкими границами, размером 0.2 ДД.

Артерии и вены имеют нормальный ход и нормальную извитость, бифуркация сосудов без особенностей, калибр в норме. Соотношение калибра артерий к венам 2/3. По ходу сосудов отмечаются единичные шарообразные утолщения стенок, окруженные очагами красного цвета с четкими границами.

Макулярный и фовеолярный рефлексы сохранены, нормальные. В области макулы дополнительно определяются кольцевидные рефлексы.

На периферии глазного дна выявляются множественные очаги красного цвета.


In [4]:
with open("train/GPT_train_prompts.json", "r", encoding="utf-8") as f:
    train_prompts = json.load(f)

In [5]:
model_str="gpt-4.5-preview-2025-02-27"

In [6]:
# Файл для логирования ответов
log_file_path = "train/train_texts.json"

# Создание пустого JSON-файла в начале
with open(log_file_path, "w", encoding="utf-8") as f:
    json.dump([], f, ensure_ascii=False, indent=2)

for prompt in tqdm(train_prompts):
    success = False
    while not success:
        try:
            response = client.responses.create(
                model=model_str,
                input=prompt
            )

            output_text = response.output_text

            with open(log_file_path, "r", encoding="utf-8") as f:
                log_data = json.load(f)

            # Добавление нового элемента
            log_data.append({
                "prompt": prompt,
                "output_text": output_text
            })

            # Запись обновлённого содержимого
            with open(log_file_path, "w", encoding="utf-8") as f:
                json.dump(log_data, f, ensure_ascii=False, indent=2)

            success = True
        except:
            time.sleep(30)


100%|██████████| 150/150 [18:44<00:00,  7.49s/it]


In [7]:
with open("train/train_texts.json", "r", encoding="utf-8") as f:
    train_texts = json.load(f)

In [8]:
df_train = pd.DataFrame(train_texts)